In [17]:

import sys,os
from Bio import Entrez,Medline
import xml.etree.cElementTree as et
from io import StringIO


In [18]:

#------- Read query file
def read_query(queryFile):
    query=[]
    with open(queryFile,'r') as q:
        for line in q:
            line = line.strip()
            query.append(line)
    return query

#------- Search related articles
def search_medline(query, email):
    Entrez.email = email
    search = Entrez.esearch(db='pubmed', term = query,retmax=5000,usehistory='y')
    handle = Entrez.read(search)
    #print(len(handle['IdList']),handle['IdList'])
    try:
        return handle
    except :
        print("Problem with query")
    finally:
        search.close()

def fetch_rec(rec_id, entrez_handle):
    fetch_handle = Entrez.efetch(db='pubmed', id=rec_id,rettype='Medline', retmode='text',
    webenv = entrez_handle['WebEnv'],
    query_key = entrez_handle['QueryKey'])
    rec = fetch_handle.read()
    return rec

In [19]:

### Programme 
query='strongyl* horse'
email='michel.boisseau@inra.fr'

rec_handler = search_medline(query, email)
print("%s\t%d" % ('Nobservations avant filtre:',len(rec_handler['IdList'])))
for k,v in rec_handler.items():
    print(k,v)
print(len(rec_handler['IdList']))

Nobservations avant filtre:	1032
Count 1032
RetMax 1032
RetStart 0
QueryKey 1
WebEnv MCID_5f60d27cc71f9352706bcccf
IdList ['32920897', '32912340', '32904743', '32801106', '32725717', '32717982', '32644306', '32448518', '32272754', '32198995', '32145074', '32095427', '32063741', '31981673', '31861066', '31690421', '31679547', '31557509', '31545964', '31491731', '31449972', '31443829', '31442894', '31422463', '31395205', '31390438', '31375025', '31303234', '31213242', '32904767', '31112466', '31084747', '31079830', '31079825', '31036059', '31014896', '31014878', '31014862', '31014805', '31014733', '31011533', '31002083', '30986403', '30981306', '30981305', '30944540', '30929787', '30850006', '30815358', '30786147', '30681130', '30364581', '30253857', '30253849', '30197013', '30197012', '30132429', '30056985', '30056984', '29984232', '29846453', '29775454', '29773232', '29717575', '29716644', '29704940', '29678224', '29651122', '29650055', '29618989', '29599559', '29432771', '29426708', '

In [20]:

outFile = str(os.getcwd() + '/' + query + '.txt')
n=0
with open(outFile, 'w') as o:
    for rec_id in rec_handler['IdList']:
        rec = fetch_rec(rec_id, rec_handler)
        rec_file = StringIO(rec)
        medline_rec = Medline.read(rec_file)
        if 'AB' in medline_rec and ('intensity' in medline_rec['AB']  or 'abundance' in medline_rec['AB'] or 'Intensity' in medline_rec['AB']  or 'Abundance' in medline_rec['AB'] or 'Prevalence' in medline_rec['AB']  or 'prevalence' in medline_rec['AB'] or 'abundance' in medline_rec['AB']or 'communit' in medline_rec['AB'] or 'Helminth' in medline_rec['AB'] or 'helminth' in medline_rec['AB'] or 'Cylicocyclus' in medline_rec['AB'] or 'Cyathostomum' in medline_rec['AB'] or 'Cylicostephanus' in medline_rec['AB']):
           # print(medline_rec['AB'])
            n+=1
            o.write(("%s\t%s\t%s\n%s\n") % (medline_rec['PMID'],medline_rec['DP'],medline_rec['TI'],medline_rec['AB']))

print("%s\t%d" % ('Nobservations avant filtre:',len(rec_handler['IdList'])))
print("%s\t%d" % ('Nobservations:',n))

Nobservations avant filtre:	1032
Nobservations:	300
